In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

## S.5 DataFrame 변환
DataFrame으로 만들어진 데이터를 변환해보자. 이러한 작업이 필요한 이유는 **기계학습에 넘겨줄 입력데이터를 형식에 맞추어야 하기 때문**이다.

데이터는 형식에 맞게 변환되고, 군집화, 회귀분석, 분류, 추천 모델 등에 입력으로 사용된다 물론 데이터는 '일련의 수' 또는 '텍스트'로 구성된다. 이런 데이터로부터 특징을 추출하여 feature vectors를 구성한다. 지도학습을 하는 경우에는 class 또는 label 값이 필요하다.

### S.5.1 Labeled Point를 label, features 컬럼으로 분해
RDD LabeledPoint는 label과 vectors로 구성되어 있다. 따라서 LabeledPoint를 DataFrame으로 읽어오면, 2개의 컬럼으로 생성된다. 이를 label, features 컬럼으로 맞추어 주도록 하자.

#### 레이블이 있는 Python List에서 DataFrame 생성
label과 features를 가지고 가지고 있는 데이터를 생성해보자.

In [2]:
p = [[1, [1.0, 2.0, 3.0]], [1, [1.1, 2.1, 3.1]], [0, [1.2, 2.2, 3.3]]]

In [3]:
print ("label: {}\nfeatures: {}".format(p[0][0], p[0][1]))

label: 1
features: [1.0, 2.0, 3.0]


In [4]:
trainDf=spark.createDataFrame(p)

In [5]:
trainDf.collect() #_1, _2라고 자동으로 생성

[Row(_1=1, _2=[1.0, 2.0, 3.0]),
 Row(_1=1, _2=[1.1, 2.1, 3.1]),
 Row(_1=0, _2=[1.2, 2.2, 3.3])]

#### LabeledPoint에서 DataFrame 생성
Python List를 LabeledPoint로 만들어 보자. LabeledPoint는 RDD에서 사용하는 구조로서 mllib 라이브러리를 사용해서 만들고 있다. DataFrame은 LabeledPoint를 컬럼으로 가지고 있지 않는다.


LabeledPoint벡터로 df을 만들면 자동으로 label과 features가 구분된다


In [6]:
from pyspark.mllib.regression import LabeledPoint
p = [LabeledPoint(1, [1.0,2.0,3.0]),
     LabeledPoint(1, [1.1,2.1,3.1]),
     LabeledPoint(0, [1.2,2.2,3.3])]

In [7]:
trainDf=spark.createDataFrame(p)

In [8]:
trainDf.collect()

[Row(features=DenseVector([1.0, 2.0, 3.0]), label=1.0),
 Row(features=DenseVector([1.1, 2.1, 3.1]), label=1.0),
 Row(features=DenseVector([1.2, 2.2, 3.3]), label=0.0)]

#### mllib.linalg.Vectors를 사용하여 DataFrame을 생성
앞서 mllib와 ml 모듈을 섞어서 사용하지 않아야 한다고 했다. mllib vectors를 사용해도 DataFrame을 생성하는데 문제가 없다. 컬럼명을 ["label", "features"]으로 하지 않으면, 자동명명되니 주의하자.

In [9]:
# LabeledPoint가 아닌 경우
from pyspark.mllib.linalg import Vectors

trainDf = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, 1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, 0.5]))], ["label", "features"]) #컬럼 명 주어야한다.

In [10]:
trainDf.collect()

[Row(label=1.0, features=DenseVector([0.0, 1.1, 0.1])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 1.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.3, 1.0])),
 Row(label=1.0, features=DenseVector([0.0, 1.2, 0.5]))]

#### RDD에서 DataFrame 생성
rdd에서 DataFrame을 생성하면 labe, features이 당연히 생성이 되지 않는다.

In [11]:
#from pyspark.mllib.linalg import SparseVector # mllib ok
from pyspark.ml.linalg import SparseVector # ml ok

_rdd = spark.sparkContext.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])

In [12]:
_df=_rdd.toDF()

In [13]:
_df.printSchema() #컬럼명 주지 않아서 자동으로 만들어

root
 |-- _1: double (nullable = true)
 |-- _2: vector (nullable = true)



In [14]:
_df=_df.withColumnRenamed('_1', 'label').withColumnRenamed('_2', 'features')

In [15]:
_df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0| (4,[1,3],[1.0,5.5])|
|  1.0|(4,[0,2],[-1.0,0.5])|
+-----+--------------------+



### S.5.2 단어 빈도
정량 데이터는 합계, 평균, 표준편차 등 의미있는 통계량을 계산하거나, 이런 통계량이 집단 간에 차이가 있는지 분석한다. 반면에 텍스트는 정량데이터와 같이 이러한 통계량의 계산이 가능하지 못하게 된다. 텍스트에 어떤 단어가 얼마나 쓰였는지, 또한 같이 쓰이게 된 단어는 무엇인지 등 단어의 빈도에 따라 정량화하여 과학적인 분석을 하게 된다.

#### Bag of Words 모델
자연어처리 NLP에서 사용하는 모델로, 텍스트를 단어의 집합, 'bag of words'으로 구성된다고 보며, 단어의 순서는 의미를 가지지 않는다. 이런 영화리뷰가 있다고 하자.

>"...그 영화는 매우 강렬했다. 그냥 좋았다. 영화관에서 보는 동안 긴장을 늦출 수 없었다. 갑돌이가 분장한 악당의 케릭터가 만들어지는 과정은 흥미롭지 않을 수가 없었다. 무비의 이야기 전개는 빠르고, 무엇이 진실이고 거짓인지 판단할 수 없었다. 누가 이런 영화를 좋아 하지 않을 수가 있겠는가 이모티콘..."

이를 단어로 분리하고, 정량화 해보자.

#### 텍스트 변환 단계
- 단계 1: 단어로 분할 Tokenization
  - 그, 영화는, 매우, 강렬했다, 그냥, 좋았다, 영화관에서, 보는, 동안, 긴장을, 늦출, 수, 없었다, 갑돌이가, 분장한, 악당의, 케릭터가, 만들어지는, 과정은, 흥미롭지, 않을, 수가, 없었다, 무비의, 이야기, 전개는, 빠르고, 무엇이, 진실이고, 거짓인지, 판단할, 수, 없었다, 누가, 이런, 영화를, 좋아, 하지, 않을, 수가, 있겠는가, 이모티콘


- 단계 2: 정리
  - 불필요, 오타, 동음이의, 이음동의 등


- 단계 3: 불용어 stopwords 제거
  - 그, 수, 수가, 수, 이런, 하지, 수가 등
  
  
- 단계 4: 어간 추출 stemming 영화는, 영화의는 다른 단어지만 조사를 제거하면 동일한 단어
  - 좋았다, 좋아 단어들은 어근을 판별하면 동일한 단어이다.
  - 영화, 무비의 단어는 이음동의
  
- 단계 5: 계량화

  - word vector로 만든다.
  - 있다-없다, 단어빈도, TF-IDF 사용할 수 있다.
dense, sparse 모두 가능하다. [1,1,1,1,1,0,0],[0,1,0,1,1,1,1]

#### Python을 사용한 단어 빈도 계산

In [16]:
# Let it be lyrics
doc=[
    "When I find myself in times of trouble",
    "Mother Mary comes to me",
    "Speaking words of wisdom, let it be",
    "And in my hour of darkness",
    "She is standing right in front of me",
    "Speaking words of wisdom, let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Whisper words of wisdom, let it be"
]

In [17]:
d={}
for sentence in doc:
    words=sentence.split()
    for word in words:
        if word in d:
            d[word]+=1
        else:
            d[word]=1

In [18]:
# for k,v in d.iteritems():  # python2
for k,v in d.items():
    print ("{}\t{}".format(k,v))

When	1
I	1
find	1
myself	1
in	3
times	1
of	6
trouble	1
Mother	1
Mary	1
comes	1
to	1
me	2
Speaking	2
words	3
wisdom,	3
let	3
it	7
be	7
And	1
my	1
hour	1
darkness	1
She	1
is	1
standing	1
right	1
front	1
Let	4
Whisper	1


### S.5.3 Spark의 transformer, estimator
RDD를 만들고 나서도 데이터를 변환하기 위해 map-reduce와 같은 함수 또는 transform(), fit()을 사용하는 것과 같이, DataFrame도 역시 Transformer, Estimator를 사용할 수 있다. 이러한 Spark ml 라이브러리는 Python의 scikit-learn에서 영향을 받아 기계학습 API transformer, estimator, evaluator가 유사하다.

- Estimator는 DataFrame에 적용되는 알고리즘을 말하는 것으로, Transformer를 생성해낸다. `Estimator.fit()`
- Transformer는 DataFrame을 적용해서 다른 DataFrame으로 생성한다. `Transformer.transform()`
- Evaluator는 pyspark.ml.evaluation의 'BinaryClassificationEvaluator', 'RegressionEvaluator', 'MulticlassClassificationEvaluator', 'MultilabelClassificationEvaluator', 'ClusteringEvaluator', 'RankingEvaluator' 등이 있다.


먼저 텍스트를 2차원 배열로 만들자.


In [19]:
doc2d=[
    ["When I find myself in times of trouble"],
    ["Mother Mary comes to me"],
    ["Speaking words of wisdom, let it be"],
    ["And in my hour of darkness"],
    ["She is standing right in front of me"],
    ["Speaking words of wisdom, let it be"],
    [u"우리 Let it be"],
    [u"나 Let it be"],
    [u"너 Let it be"],
    ["Let it be"],
    ["Whisper words of wisdom, let it be"]
]

그리고 DataFrame을 생성한다. schema는 만들어 주지 않고 컬럼명을 sent로 한다.

In [20]:
myDf=spark.createDataFrame(doc2d, ['sent'])

truncate=True는 줄여서, False는 출력을 줄이지 않고 출력한다.

In [21]:
myDf.show(truncate=True)

+--------------------+
|                sent|
+--------------------+
|When I find mysel...|
|Mother Mary comes...|
|Speaking words of...|
|And in my hour of...|
|She is standing r...|
|Speaking words of...|
|      우리 Let it be|
|        나 Let it be|
|        너 Let it be|
|           Let it be|
|Whisper words of ...|
+--------------------+



In [22]:
myDf.show(truncate=False)

+--------------------------------------+
|sent                                  |
+--------------------------------------+
|When I find myself in times of trouble|
|Mother Mary comes to me               |
|Speaking words of wisdom, let it be   |
|And in my hour of darkness            |
|She is standing right in front of me  |
|Speaking words of wisdom, let it be   |
|우리 Let it be                        |
|나 Let it be                          |
|너 Let it be                          |
|Let it be                             |
|Whisper words of wisdom, let it be    |
+--------------------------------------+



### S.5.4 Tokenizer

토큰: 최소단위의 상징

먼저 용어를 정리해 보자.

- corpus는 어떤 주제에 대해 쓰여지거나, 어떤 사람이 작성한 전체 '말뭉치'를 말한다. 여러 문장으로 구성된 텍스트 집합을 말한다.


- document는 문장으로 구성된 문서를 말하지만, 한 문장으로만 구성될 수도, 여러 문장으로 만들어질 수도 있다. 예를 들어, "why she had to go" 같은 한 문장도 document라고 하고, "why she had to go.. I don't know" 역시 마찬가지이다.


- vocabularay는 중복없는 단어 집합을 말하며, 예를 들면, "why","she","had","to","go","where","have" 등은 단어이다.


- Tokenizer는 document를 단어로 분리한다. 분리하는 기준은 whitespace로 공백, TAB, CR, New Line 등이 해당된다.

  - 입력 컬럼은 "sent"로,
  - 출력 컬럼은 "words"로 한다.

In [23]:
from pyspark.ml.feature import Tokenizer #df니까 ml이용
tokenizer = Tokenizer(inputCol="sent", outputCol="words")

transform()은 앞서 만든 tokenizer모델에 DataFrame을 변환하여 다른 DataFrame을 생성한다. 그 결과는 문자열 배열로 구성된다.

In [24]:
tokDf = tokenizer.transform(myDf) #토큰화한 df

In [25]:
tokDf.show(3)

+--------------------+--------------------+
|                sent|               words|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
+--------------------+--------------------+
only showing top 3 rows



for문으로 출력해보자. Row() 객체로 출력된다.

In [26]:
for r in tokDf.select("sent", "words").take(3):
    print (r)

Row(sent='When I find myself in times of trouble', words=['when', 'i', 'find', 'myself', 'in', 'times', 'of', 'trouble'])
Row(sent='Mother Mary comes to me', words=['mother', 'mary', 'comes', 'to', 'me'])
Row(sent='Speaking words of wisdom, let it be', words=['speaking', 'words', 'of', 'wisdom,', 'let', 'it', 'be'])


### S.5.5 RegTokenizer (*reg*)
Tokenizer는 white space로 분리하지만, RegexTokenizer는 단어를 분리하기 위해 정규표현식을 적용할 수 있다. 정규표현식을 사용하여 분리하거나 특정 패턴을 추출할 수 있다. 공백으로 분리할 경우 간단히 정규표현식 \s 패턴을 적용할 수 있다. 한글에는 \w 패턴이 적용되지 않는다.

- \s는 공백문자
- \w는 숫자 및 대소문자 [A-Za-z0-9_]
- 별표 *는 0 또는 그 이상, 더하기 +는 1 또는 그 이상을 의미한다.

In [27]:
from pyspark.ml.feature import RegexTokenizer
re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+") #공백이 1또는 그이상일 때 분리

In [28]:
reDf=re.transform(myDf)
reDf.show()

+--------------------+--------------------+
|                sent|            wordsReg|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|
|        나 Let it be|   [나, let, it, be]|
|        너 Let it be|   [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



### S.5.6 Stopwords
텍스트를 분리하고 나면, 별 의미가 없거나 쓸모가 없는 단어들이 존재한다. 예를 들어 이, 그, 저와 같은 한 단어 또는 있다 등과 같은 일부 동사, 그래서, 그러나 등과 같은 접속사 등이 후보가 될 수 있다. 이런 불필요한 단어들을 불용어 Stopwords라고 하며, 입력데이터에서 제거하도록 한다. 영어의 경우 불용어가 식별되어 제공되고 있다 http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words

In [29]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

In [30]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

In [31]:
_mystopwords=[u"나",u"너", u"우리"] # u: unicode
for e in _mystopwords:
    stopwords.append(e) #영어불용어+우리나라불용어
stop.setStopWords(stopwords) 

StopWordsRemover_bc2ec952f014

In [32]:
for e in stop.getStopWords():
    print (e, end="/")

i/me/my/myself/we/our/ours/ourselves/you/your/yours/yourself/yourselves/he/him/his/himself/she/her/hers/herself/it/its/itself/they/them/their/theirs/themselves/what/which/who/whom/this/that/these/those/am/is/are/was/were/be/been/being/have/has/had/having/do/does/did/doing/a/an/the/and/but/if/or/because/as/until/while/of/at/by/for/with/about/against/between/into/through/during/before/after/above/below/to/from/up/down/in/out/on/off/over/under/again/further/then/once/here/there/when/where/why/how/all/any/both/each/few/more/most/other/some/such/no/nor/not/only/own/same/so/than/too/very/s/t/can/will/just/don/should/now/i'll/you'll/he'll/she'll/we'll/they'll/i'd/you'd/he'd/she'd/we'd/they'd/i'm/you're/he's/she's/it's/we're/they're/i've/we've/you've/they've/isn't/aren't/wasn't/weren't/haven't/hasn't/hadn't/don't/doesn't/didn't/won't/wouldn't/shan't/shouldn't/mustn't/can't/couldn't/cannot/could/here's/how's/let's/ought/that's/there's/what's/when's/where's/who's/why's/would/나/너/우리/

In [33]:
stopDf=stop.transform(reDf) #불용어 제거 된 컬럼
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|
|She is standing r...|[she, is, standin...|[standing, right,...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|               [let]|
|        나 Let it be|   [나, let, it, be]|               [let]|
|        너 Let it be|   [너, let, it, be]|               [let]|
|           Let it be|       [let, it, be]|               [let]|
|Whisper words of ...|[whisper, words, ...|[whisper, words, ...|
+--------------------+--------------------+--------------------+



### S.5.7 CountVectorizer
CountVectorizer는 단어의 빈도 수를 계산한다.

3번째 문장 "Speaking words of wisdom, let it be"의 word vector를 구성해 본다. id 값은 모든 문장에서 단어를 추출하고 나서야 부여된다.

단어 (3행 "Speaking words of wisdom, let it be")|id|빈도
:---|---|---
Speaking|7|1
words|13|1
of|stopword|0
wisdom|12|1
let|3|1
it|stopword|0
be|stopword|0

위 word vector를 표로 나타내면 아래와 같다. 행은 문장, 열은 id이다. 3행은 doc2이다. 해당하는 단어 id의 빈도를 적었다. 다른 행과 열은 이해를 돕기 위해 비워 놓았다.

doc \ 단어 id|1|2|3|4|5|6|7|8|9|10|11|12|13|...
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
doc 0||||||||||||||...
doc 1||||||||||||||...
doc 2|||1||||1|||||1|1|...
...||||||||||||||...

#### sklearn CountVectorizer
sklearn 라이브러리로 단어빈도를 세어보자. 입력으로 단어의 집합을 넣어야 하지만, 위 문서는 2차원 리스트이다. 먼저 2차원 리스트를 1차원으로 변경하자.

In [34]:
from functools import reduce
doc = reduce(lambda x,y: x+y, doc2d)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english') #내장 stopwords

In [36]:
print (vectorizer.fit_transform(doc))

  (0, 9)	1
  (0, 10)	1
  (1, 5)	1
  (1, 4)	1
  (1, 0)	1
  (2, 7)	1
  (2, 13)	1
  (2, 12)	1
  (2, 3)	1
  (3, 2)	1
  (3, 1)	1
  (4, 8)	1
  (4, 6)	1
  (5, 7)	1
  (5, 13)	1
  (5, 12)	1
  (5, 3)	1
  (6, 3)	1
  (6, 14)	1
  (7, 3)	1
  (8, 3)	1
  (9, 3)	1
  (10, 13)	1
  (10, 12)	1
  (10, 3)	1
  (10, 11)	1


In [37]:
vectorizer.vocabulary_

{'times': 9,
 'trouble': 10,
 'mother': 5,
 'mary': 4,
 'comes': 0,
 'speaking': 7,
 'words': 13,
 'wisdom': 12,
 'let': 3,
 'hour': 2,
 'darkness': 1,
 'standing': 8,
 'right': 6,
 '우리': 14,
 'whisper': 11}

In [38]:
vectorizer.fit_transform(doc).todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]], dtype=int64)

#### Spark CountVectorizer
- 입력: text, Tockenizer를 사용해서 단어로 분리하고 난 후 사용
- 출력: 단어 빈도 word vector(sparse), 즉 단어별 단어 빈도 TF
- minDF은 자주 사용된 단어가 아니어서 제거해야할 단어, 즉 문서에 사용된 빈도 document frequency를 제한하는 것. 소수점 사용해야함(비율을 의미하기 때문)
  - ex) minDF=1.0 : 문서 1개 이하에서 나타난 단어는 무시
- maxDf는 너무 많이 발생하는 경우 무시
  - ex) 0.5는 전체 문서의 50%보다 많이 발생하는 경우 무시, 1.0은 100%보다 많이 발생하는 경우 무시 (즉, 어떤 단어도 무시하지 말라는 의미)
  
  정수는 사용된 문서의 수, 몇 개의 문서에 사용되어야하는지

In [39]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=30, minDF=1.0)

CountVectorizerModel은 fit()하고 나면 얻어진다. 다음에 사용하는 HashingTF는 fit()하지 않는다는 점에서 차이가 있다.

In [40]:
cvModel = cv.fit(stopDf)

In [41]:
print (type(cv),type(cvModel))

<class 'pyspark.ml.feature.CountVectorizer'> <class 'pyspark.ml.feature.CountVectorizerModel'>


In [42]:
cvDf = cvModel.transform(stopDf)

In [43]:
cvDf.show(3)

+--------------------+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|                  cv|
+--------------------+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|(16,[5,6,8],[1.0,...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|(16,[10,13,14],[1...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|(16,[0,1,2,3],[1....|
+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



DataFrame 전체를 출력하면 보기 불편하므로, 이 가운데 일부 컬럼만을 선택하여 출력할 수 있다. (16,[5,6,8],[1.0,1.0,1.0]) 16은 전체 단어의 개수, 그리고 다음 5,6,8은 값이 있는 컬럼 번호, 1.0,1.0,1.0은 그 값을 말한다.

In [44]:
cvDf.select('sent','nostops','cv').show()

+--------------------+--------------------+--------------------+
|                sent|             nostops|                  cv|
+--------------------+--------------------+--------------------+
|When I find mysel...|[find, times, tro...|(16,[5,6,8],[1.0,...|
|Mother Mary comes...|[mother, mary, co...|(16,[10,13,14],[1...|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|And in my hour of...|    [hour, darkness]|(16,[7,9],[1.0,1.0])|
|She is standing r...|[standing, right,...|(16,[4,12,15],[1....|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|      우리 Let it be|               [let]|      (16,[0],[1.0])|
|        나 Let it be|               [let]|      (16,[0],[1.0])|
|        너 Let it be|               [let]|      (16,[0],[1.0])|
|           Let it be|               [let]|      (16,[0],[1.0])|
|Whisper words of ...|[whisper, words, ...|(16,[0,1,2,11],[1...|
+--------------------+--------------------+--------------------+



CountVectorizer에서 사용된 단어 목록을 출력할 수 있다. 아래 단어의 수를 세어보면 위 sparse vector의 컬럼 개수와 동일하다.

In [45]:
cvModel.vocabulary

['let',
 'wisdom,',
 'words',
 'speaking',
 'right',
 'trouble',
 'find',
 'hour',
 'times',
 'darkness',
 'mother',
 'whisper',
 'front',
 'mary',
 'comes',
 'standing']

### S.5.8 TF-IDF
TfidfTransformer는 **TF-IDF(Term Frequency-Inverse Document Frequency)** 를 계산한다. 이를 위해서는 우선 Tokenizer를 사용하여 문장을 단어로 분리해 놓아야 한다.

HashingTF를 사용하여 'word vector'를 계산한다. HashingTF은 hash함수에 따라 단어의 고유번호를 생성하며, 단어 수가 많아지면서 **고유번호가 충돌할 수 있는 가능성이 있지만 이를 최소화**하게 된다. 그리고 IDF를 계산하고, TF-IDF를 계산한다.

#### S.5.8.1 TF-IDF 계산
'Let it be'가사 세 번째 줄 'wisdom' 단어의 TF-IDF를 계산해보자  
TF는 단어빈도수, 즉 문서에 단어가 나타난 빈도수를 의미한다. 단어빈도는 경우에 따라서는 문제가 될 수 있다. 예를 들어, 'a', 'the', 'of'와 같은 단어는 빈도는 높지만 별로 유용하지 못하다.

이 경우 **IDF**는 유용하다. IDF는 자주 나타나는 단어에 대한 가중치를 줄이고, 드물게 나타나는 단어에 가중치를 높이는 방식으로 계산된다.


t는 단어, 문서는 d, D는 corpus,

항목|설명|예제
:---|:---|:---
tf(d,f)|단어 t가 문서 d에서 나타나는 단어의 빈도 수, term frequency|$f_{t,d}$ / (number of words in d) = 1/4 = 0.25  (3번째 문서에 stopwords를 제외하면 4개의 단어, wisdom은 1회 나타난다.)
df|document frequency 단어가 나타난 문서 수|3 (wisdom이 포함된 문서는 3)
N|number of documents 전체 문서의 수|11 (전체의 문서는 11개)
idf|inverse document frequency 단어가 나타난 문서의 비율을 거꾸로|ln(N+1 / df+1) + 1 = log(12/4) + 1 = 1.09861 + 1, 0으로 나뉘는 것을 방지하기 위해 smoothing, 즉 1을 더한다.


프로그래밍에서는 메모리를 적게 사용하도록 설계되어 있다. 1/4와 같이 정수 타잎으로 연산하면, 정수를 사용하여 연산하고 그 결과도 정수를 출력하게 된다. 이를 변환하기 위해 1.의 경우에서와 같이 소수를 사용하자.

In [46]:
import math

tf=1./4
df=3.
N=11.
idf=math.log((N+1)/(df+1))+1
print ("idf: {}".format(idf))

idf: 2.09861228866811


#### S.5.8.2 sklearn을 사용한 TF-IDF
우선 'sklearn'의 TF-IDF를 계산해보자. CountVectorizer는 텍스트를 단어의 빈도로 변환해주어, 문서 x 단어 표를 출력할 수 있다. CountVectorizer()의 인자로 analyzer ("word", "character ngram" 등 선택), tokenizer (단어의 tokenizer를 지정), stop_words (불용어 처리 기준), max_features (최대 속성 개수) 등을 지정할 수 있다. 그 다음으로, TF-IDF를 계산할 수 있다. 이 때 (문서id, 단어id) 별로 결과가 출력된다.

TfidfVectorizer를 사용해서 계산하면 그 결과를 아래와 같이 볼 수 있다.

>(2,12) 2.09861228867

결과에서 '2'는 3번째 문서번호, '12'는 'wisdom' 단어번호 TF-IDF는 2.09861228867이다.



max_df, min_df는 기본 값이 1.0으로 굳이 설정하지 않아도 되는데, **어떤 단어도 무시하지 말라는 의미**이다.

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0, stop_words='english',norm = None)

In [48]:
print (vectorizer.fit_transform(doc))

  (0, 10)	2.791759469228055
  (0, 9)	2.791759469228055
  (1, 0)	2.791759469228055
  (1, 4)	2.791759469228055
  (1, 5)	2.791759469228055
  (2, 3)	1.4054651081081644
  (2, 12)	2.09861228866811
  (2, 13)	2.09861228866811
  (2, 7)	2.386294361119891
  (3, 1)	2.791759469228055
  (3, 2)	2.791759469228055
  (4, 6)	2.791759469228055
  (4, 8)	2.791759469228055
  (5, 3)	1.4054651081081644
  (5, 12)	2.09861228866811
  (5, 13)	2.09861228866811
  (5, 7)	2.386294361119891
  (6, 14)	2.791759469228055
  (6, 3)	1.4054651081081644
  (7, 3)	1.4054651081081644
  (8, 3)	1.4054651081081644
  (9, 3)	1.4054651081081644
  (10, 11)	2.791759469228055
  (10, 3)	1.4054651081081644
  (10, 12)	2.09861228866811
  (10, 13)	2.09861228866811


In [49]:
vectorizer.vocabulary_

{'times': 9,
 'trouble': 10,
 'mother': 5,
 'mary': 4,
 'comes': 0,
 'speaking': 7,
 'words': 13,
 'wisdom': 12,
 'let': 3,
 'hour': 2,
 'darkness': 1,
 'standing': 8,
 'right': 6,
 '우리': 14,
 'whisper': 11}

In [50]:
vectorizer.idf_

array([2.79175947, 2.79175947, 2.79175947, 1.40546511, 2.79175947,
       2.79175947, 2.79175947, 2.38629436, 2.79175947, 2.79175947,
       2.79175947, 2.79175947, 2.09861229, 2.09861229, 2.79175947])

#### S.5.8.3 Spark를 사용한 TF-IDF
##### TF
HashingTF는 단어집합을 워드벡터 word vector로 변환하는데, 해시함수를 사용해서 단어에 해당하는 일련번호를 결정한다. HashingTF에서의 numFeatures는 $2^n$으로 결정하게 된다. 단어 갯수가 900이면, $2^{10}=1024$이므로 1024로 설정하면 된다. 기본은 $2^{18}=262,144$이다. **너무 적게 설정되면 인덱스가 부족하거나 적절하게 매핑될 수 있으니 주의해야 한다.**



예를 들어, 문서 [speaking, words, wisdom,, let]의 경우 (32,[4,24,27],[1.0,1.0,2.0])가 출력된다. 아래의 결과와 비교하면 단어 하나가 유실된 것을 알 수 있다. (4개가 나타나야함)  
32를 생략하면 기본값 (2^18)

In [51]:
from pyspark.ml.feature import HashingTF, IDF

# hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=32) #  mapping indices insufficient
hashTF = HashingTF(inputCol="nostops", outputCol="hash")

In [52]:
hashDf = hashTF.transform(stopDf)

In [53]:
hashDf.select("nostops", "hash").show(truncate=False)

+-------------------------------+--------------------------------------------------------+
|nostops                        |hash                                                    |
+-------------------------------+--------------------------------------------------------+
|[find, times, trouble]         |(262144,[64317,91878,152481],[1.0,1.0,1.0])             |
|[mother, mary, comes]          |(262144,[24657,63767,245426],[1.0,1.0,1.0])             |
|[speaking, words, wisdom,, let]|(262144,[27556,151864,173339,175131],[1.0,1.0,1.0,1.0]) |
|[hour, darkness]               |(262144,[74517,98431],[1.0,1.0])                        |
|[standing, right, front]       |(262144,[84798,218360,229166],[1.0,1.0,1.0])            |
|[speaking, words, wisdom,, let]|(262144,[27556,151864,173339,175131],[1.0,1.0,1.0,1.0]) |
|[let]                          |(262144,[173339],[1.0])                                 |
|[let]                          |(262144,[173339],[1.0])                                 |

##### TF-IDF

In [54]:
idf = IDF(inputCol="hash", outputCol="idf")

In [55]:
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [56]:
for e in idfDf.select("nostops","hash").take(10):
    print(e)

Row(nostops=['find', 'times', 'trouble'], hash=SparseVector(262144, {64317: 1.0, 91878: 1.0, 152481: 1.0}))
Row(nostops=['mother', 'mary', 'comes'], hash=SparseVector(262144, {24657: 1.0, 63767: 1.0, 245426: 1.0}))
Row(nostops=['speaking', 'words', 'wisdom,', 'let'], hash=SparseVector(262144, {27556: 1.0, 151864: 1.0, 173339: 1.0, 175131: 1.0}))
Row(nostops=['hour', 'darkness'], hash=SparseVector(262144, {74517: 1.0, 98431: 1.0}))
Row(nostops=['standing', 'right', 'front'], hash=SparseVector(262144, {84798: 1.0, 218360: 1.0, 229166: 1.0}))
Row(nostops=['speaking', 'words', 'wisdom,', 'let'], hash=SparseVector(262144, {27556: 1.0, 151864: 1.0, 173339: 1.0, 175131: 1.0}))
Row(nostops=['let'], hash=SparseVector(262144, {173339: 1.0}))
Row(nostops=['let'], hash=SparseVector(262144, {173339: 1.0}))
Row(nostops=['let'], hash=SparseVector(262144, {173339: 1.0}))
Row(nostops=['let'], hash=SparseVector(262144, {173339: 1.0}))


### S.5.9 Word2Vec
단어를 벡터로 변환하는 방법을 말한다.

**Bag of Words** 모델은 **단어 순서와 문맥을 무시**한다.  
**Word2Vec**는 이런 BoW 모델의 단점을 극복하기 위해서, **말뭉치로부터 단어들 서로의 맥락 또는 연관성 Word Embedding을 신경망으로 학습하여 Word2Vec을 계산**한다. 단어가 벡터로 변환되면, 벡터연산이 가능해지고 서로 간의 거리를 측정하여, 가까울수록 비슷한 단어라고 해석하게 된다.  

Word2Vec을 계산하고 나면, 벡터('king') - 벡터('man') + 벡터('woman') = 벡터('queen) 이런 연산이 가능해진다. 즉 king 단어벡터에서 man 단어백터를 빼고, woman 단어백터를 더하면, queen 단어백터를 구할 수 있다는 의미이다.

vectorSize는 단어벡터를 몇 개로 구성할 것인지, minCount는 최소 단어빈도를 설정할 수 있다.

In [57]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="w2v")
model = word2Vec.fit(tokDf) #토큰으로 분리시킨 다음에 적용시켜야한다.
w2vDf = model.transform(tokDf)

In [58]:
for e in w2vDf.select("w2v").take(3):
    print(e)

Row(w2v=DenseVector([-0.0426, -0.0448, -0.0308]))
Row(w2v=DenseVector([0.0002, -0.0528, 0.0787]))
Row(w2v=DenseVector([-0.0691, -0.0187, 0.0205]))


In [59]:
model.getVectors().show(truncate=False) #숫자: 신경망을 통해서 생성

+--------+------------------------------------------------------------------+
|word    |vector                                                            |
+--------+------------------------------------------------------------------+
|trouble |[-0.10907474905252457,-0.12260671705007553,-0.008148939348757267] |
|mother  |[-0.089715875685215,0.04702672362327576,0.026056621223688126]     |
|find    |[-0.0019626470748335123,0.027025384828448296,-0.16112865507602692]|
|standing|[-0.1180938333272934,0.014508146792650223,0.1571418046951294]     |
|wisdom, |[-0.15978337824344635,0.07689274847507477,0.033270757645368576]   |
|in      |[-0.07372022420167923,-0.08980047702789307,-0.035899002104997635] |
|myself  |[-0.06915149837732315,-0.008410882204771042,-0.15042924880981445] |
|is      |[-0.1075735092163086,0.11674908548593521,-0.026824142783880234]   |
|darkness|[0.11327086389064789,-0.12288354337215424,0.02928961254656315]    |
|우리    |[-0.058154866099357605,-0.02477012760937214,-0.093219600

In [60]:
model.findSynonyms("times", 3).show() #문장에서 비어있는 단어 채우는 등의 일을 할 수 있다.

+--------+------------------+
|    word|        similarity|
+--------+------------------+
|      my|0.9588775038719177|
|darkness|0.8516458868980408|
|      너|0.7743990421295166|
+--------+------------------+



### S.5.10 NGram
단어 n 개가 붙어 있을 때 더 의미가 있는 경우가 있다.
텍스트를 대상으로 하면, n-gram은 연속된 n개의 토큰으로 구성된 순열을 말한다.  
unigram은 한 단어로, bigram은 두 단어로 구성한다.

In [61]:
from pyspark.ml.feature import NGram

ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

In [62]:
ngramDf = ngram.transform(tokDf) #토큰으로 분리시킨 다음에 적용시켜야한다.

In [63]:
ngramDf.show() #i find 와 같이 붙을 수도 있다.

+--------------------+--------------------+----------------------+
|                sent|               words|                ngrams|
+--------------------+--------------------+----------------------+
|When I find mysel...|[when, i, find, m...|  [when i, i find, ...|
|Mother Mary comes...|[mother, mary, co...|  [mother mary, mar...|
|Speaking words of...|[speaking, words,...|  [speaking words, ...|
|And in my hour of...|[and, in, my, hou...|  [and in, in my, m...|
|She is standing r...|[she, is, standin...|  [she is, is stand...|
|Speaking words of...|[speaking, words,...|  [speaking words, ...|
|      우리 Let it be| [우리, let, it, be]|[우리 let, let it, ...|
|        나 Let it be|   [나, let, it, be]| [나 let, let it, i...|
|        너 Let it be|   [너, let, it, be]| [너 let, let it, i...|
|           Let it be|       [let, it, be]|       [let it, it be]|
|Whisper words of ...|[whisper, words, ...|  [whisper words, w...|
+--------------------+--------------------+----------------------+



In [64]:
for e in ngramDf.select("words","ngrams").take(3):
    print (e)

Row(words=['when', 'i', 'find', 'myself', 'in', 'times', 'of', 'trouble'], ngrams=['when i', 'i find', 'find myself', 'myself in', 'in times', 'times of', 'of trouble'])
Row(words=['mother', 'mary', 'comes', 'to', 'me'], ngrams=['mother mary', 'mary comes', 'comes to', 'to me'])
Row(words=['speaking', 'words', 'of', 'wisdom,', 'let', 'it', 'be'], ngrams=['speaking words', 'words of', 'of wisdom,', 'wisdom, let', 'let it', 'it be'])


### S.5.11 StringIndexer
문자열 컬럼을 인덱스 컬럼으로 변환한다. 빈도가 제일 높은 순서로 0.0부터 인덱스 값이 주어진다. 인덱스는 double 형을 가지게 된다. 없는 레이블에 대해서는 예외가 발생할 수 있으므로 (default), setHandleInvalid("skip") 함수로 'skip', 'keep', 'error' 등으로 설정할 수 있다.

label, 명목변수: 단어 그 자체로 의미가 있다.


구분|설명|예
:---:|:---:|:---:
nominal|명목 또는 구분 값 cateogry|사자, 호랑이, 사람
ordinal|명목값과 다른 점은 순서가 있다.|키 low, med, high
interval|일정한 간격이 있다.|150-165, 165-180, 180-195


현재 텍스트에 대해서는 적당한 명목변수가 없으므로, 문장 전체에 대해 인덱스로 변환해보자.

In [65]:
from pyspark.ml.feature import StringIndexer

labelIndexer = StringIndexer(inputCol="sent", outputCol="sentLabel") #문장 전체를 넣었음.

In [66]:
model=labelIndexer.fit(myDf)

In [67]:
siDf=model.transform(myDf)

In [68]:
siDf.show()

+--------------------+---------+
|                sent|sentLabel|
+--------------------+---------+
|When I find mysel...|      5.0|
|Mother Mary comes...|      3.0|
|Speaking words of...|      0.0|
|And in my hour of...|      1.0|
|She is standing r...|      4.0|
|Speaking words of...|      0.0|
|      우리 Let it be|      9.0|
|        나 Let it be|      7.0|
|        너 Let it be|      8.0|
|           Let it be|      2.0|
|Whisper words of ...|      6.0|
+--------------------+---------+



### S.5.12 연속데이터의 변환
몸무게(inches), 키(pounds) 데이터를 분석해보자. 이 데이터는 정량, 연속 데이터이다. 출처는 https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv

In [69]:
import os
from pyspark.sql.types import *

rdd=spark.sparkContext\
    .textFile(os.path.join('data','ds_spark_heightweight.txt'))

In [70]:
myRdd=rdd.map(lambda line:[float(x) for x in line.split('\t')])

In [71]:
myDf=spark.createDataFrame(myRdd,["id","weight","height"])

In [72]:
myDf.printSchema()

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)



In [73]:
from pyspark.ml.feature import Binarizer #이분화

binarizer = Binarizer(threshold=68.0, inputCol="weight", outputCol="weight2") #68기준 이분화

In [74]:
binDf = binarizer.transform(myDf)

In [75]:
binDf.show(10) #68보다작으면 0, 크면 1이 나옴

+----+------+------+-------+
|  id|weight|height|weight2|
+----+------+------+-------+
| 1.0| 65.78|112.99|    0.0|
| 2.0| 71.52|136.49|    1.0|
| 3.0|  69.4|153.03|    1.0|
| 4.0| 68.22|142.34|    1.0|
| 5.0| 67.79| 144.3|    0.0|
| 6.0|  68.7| 123.3|    1.0|
| 7.0|  69.8|141.49|    1.0|
| 8.0| 70.01|136.46|    1.0|
| 9.0|  67.9|112.37|    0.0|
|10.0| 66.78|120.67|    0.0|
+----+------+------+-------+
only showing top 10 rows



In [76]:
from pyspark.ml.feature import QuantileDiscretizer#삼분화

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="height", outputCol="height3")

In [77]:
qdDf = discretizer.fit(binDf).transform(binDf)

In [78]:
qdDf.show(10)

+----+------+------+-------+-------+
|  id|weight|height|weight2|height3|
+----+------+------+-------+-------+
| 1.0| 65.78|112.99|    0.0|    0.0|
| 2.0| 71.52|136.49|    1.0|    1.0|
| 3.0|  69.4|153.03|    1.0|    2.0|
| 4.0| 68.22|142.34|    1.0|    2.0|
| 5.0| 67.79| 144.3|    0.0|    2.0|
| 6.0|  68.7| 123.3|    1.0|    0.0|
| 7.0|  69.8|141.49|    1.0|    2.0|
| 8.0| 70.01|136.46|    1.0|    1.0|
| 9.0|  67.9|112.37|    0.0|    0.0|
|10.0| 66.78|120.67|    0.0|    0.0|
+----+------+------+-------+-------+
only showing top 10 rows



### S.5.13 VectorAssembler
열을 묶어서 Vector열로 만든다. features 컬럼을 생성할 경우에 사용한다. 단 문자열 string은 묶을 수 없다.
기계학습- lable, features 컬럼 있음, 이 때 속성을 여러 컬럼에서 가지고 있을 수 있는데 이 경우 여러 컬럼을 합쳐서 하나의 vector 컬럼으로 만들어줌

In [79]:
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["weight2","height3"],outputCol="features")

In [80]:
vaDf = va.transform(qdDf)

In [81]:
vaDf.printSchema()  #벡터 형태 features가 만들어짐

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- weight2: double (nullable = true)
 |-- height3: double (nullable = true)
 |-- features: vector (nullable = true)



In [82]:
vaDf.show(5)

+---+------+------+-------+-------+---------+
| id|weight|height|weight2|height3| features|
+---+------+------+-------+-------+---------+
|1.0| 65.78|112.99|    0.0|    0.0|(2,[],[])|
|2.0| 71.52|136.49|    1.0|    1.0|[1.0,1.0]|
|3.0|  69.4|153.03|    1.0|    2.0|[1.0,2.0]|
|4.0| 68.22|142.34|    1.0|    2.0|[1.0,2.0]|
|5.0| 67.79| 144.3|    0.0|    2.0|[0.0,2.0]|
+---+------+------+-------+-------+---------+
only showing top 5 rows



### S.5.14 Pipeline¶
Pipeline은 여러 Estimator를 묶은 Estimator를 반환한다. Pipeline은 여러 작업을 묶어, 순서대로 단계적으로 Estimator를 적용하기 위해 사용한다.

In [83]:
df = spark.createDataFrame([
        (0, "a b c d e spark", 1.0),
        (1, "b d", 0.0),
        (2, "spark f g h", 1.0),
        (3, "hadoop mapreduce", 0.0),
        (4, "my dog has flea problems. help please.",0.0)
    ], ["id", "text", "label"])

In [84]:
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [89]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [90]:
model = pipeline.fit(df)

In [91]:
myDf = model.transform(df)

In [92]:
myDf.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262144,[74920,89...|
|  0.0|(262144,[89530,14...|
|  1.0|(262144,[36803,17...|
|  0.0|(262144,[132966,1...|
|  0.0|(262144,[1074,389...|
+-----+--------------------+



####

###